In [14]:
import os
path_to_data = "/Users/pathakav/LLM Code/vfs_data"
all_files = os.listdir(path_to_data)


In [8]:
import openai

# get api key from platform.openai.com
#openai.api_key = os.getenv('OPENAI_API_KEY') or 'OPENAI_API_KEY'
openai.api_key = "<open-ai key>"
embed_model = "text-embedding-ada-002"

In [115]:
text_dict = {}
seq_length = 300

In [161]:
i = 1.0

text_dict_size = 0

for text_file in all_files:
    print("Processing file ", text_file)
    with open(os.path.join(path_to_data,text_file), 'r') as f:
        file_content = f.read()
        
        print("length of file_content = ", len(file_content))
        
        k = 0
        while k < len(file_content):
            smaller_seq = file_content[k : k + seq_length]
            
            print("length of smaller_seq = ", len(smaller_seq))
            
            
            embedded_seq_with_metadata = openai.Embedding.create(input=[smaller_seq], engine=embed_model)
            
            embedded_seq = embedded_seq_with_metadata['data'][0]['embedding']
            
            print("length of embedding_seq = ", len(embedded_seq))
            
            text_dict[str(i)+"_text"] = smaller_seq
            text_dict[str(i)+"_embedding"] = embedded_seq
        
            i = i + 1
            k = k + seq_length
            
text_dict_size = i
        

Processing file  Terms & Conditions _ vfsglobal.txt
length of file_content =  11926
length of smaller_seq =  500
length of embedding_seq =  1536
length of smaller_seq =  500
length of embedding_seq =  1536
length of smaller_seq =  500
length of embedding_seq =  1536
length of smaller_seq =  500
length of embedding_seq =  1536
length of smaller_seq =  500
length of embedding_seq =  1536
length of smaller_seq =  500
length of embedding_seq =  1536
length of smaller_seq =  500
length of embedding_seq =  1536
length of smaller_seq =  500
length of embedding_seq =  1536
length of smaller_seq =  500
length of embedding_seq =  1536
length of smaller_seq =  500
length of embedding_seq =  1536
length of smaller_seq =  500
length of embedding_seq =  1536
length of smaller_seq =  500
length of embedding_seq =  1536
length of smaller_seq =  500
length of embedding_seq =  1536
length of smaller_seq =  500
length of embedding_seq =  1536
length of smaller_seq =  500
length of embedding_seq =  1536
l

In [162]:
text_dict

{'1.0_text': "\nGermany Visa Application Centre, Coimbatore will be opened, please click here\nDismiss \n\n\n\n\nHome arrow_forward Terms & Conditions \nTerms and conditions \nIn applying for your visa, VFS Global may provide the services as detailed below. These services will be provided were permitted by the Government of the Country you are applying to visit. These services may not be possible in all countries.\n\t•\tSHORT MESSAGING SERVICES\u2028The Short Messaging Service ('SMS') where provided to update the visa applica",
 '1.0_embedding': [-0.01791176199913025,
  0.006645562592893839,
  -0.012034039944410324,
  -0.015451951883733273,
  -0.04740229249000549,
  0.03527991846203804,
  -0.039384130388498306,
  -0.021649040281772614,
  0.0012655786704272032,
  -0.02743842639029026,
  0.01660711131989956,
  0.010049884207546711,
  -0.013624082319438457,
  -0.03147468715906143,
  0.00013016774028073996,
  0.008629717864096165,
  0.023972949013113976,
  -0.014582185074687004,
  0.015968

In [118]:
#embedded_seq

In [119]:
#print((embedded_file['data'][0]['embedding']))

In [120]:
#embedded_file

In [245]:
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
def get_context(query_embedding):

    result_list = []
    for k in range(1, int(text_dict_size)):
        #similarity_score = 1- spatial.distance.cosine(query_embedding, text_dict[str(k)+".0"+"_embedding"])
        similarity_score = cosine_similarity([query_embedding], [text_dict[str(k)+".0"+"_embedding"]])
        #print("current similarity score = ", similarity_score)
        result_list.append([similarity_score, str(k)+".0"+"_embedding"])


    result_list.sort(reverse=True, key = lambda k: k[0])

    top_1_similarity_score, top_1_embedding_id = result_list[0]
    top_2_similarity_score, top_2_embedding_id = result_list[1]
    top_3_similarity_score, top_3_embedding_id = result_list[2]
    top_4_similarity_score, top_4_embedding_id = result_list[3]
    
    
    index1 = top_1_embedding_id.strip("_")[0]
    index2 = top_2_embedding_id.strip("_")[0]
    index3 = top_3_embedding_id.strip("_")[0]
    index4 = top_4_embedding_id.strip("_")[0]
    
    text_key1 = index1 +".0"+"_text"
    text_key2 = index2 +".0"+"_text"
    text_key3 = index3 +".0"+"_text"
    text_key4 = index4 +".0"+"_text"
    
#     print("********************************************************************")
#     print("Found similar text as contexts ", text_key1, " ", text_key2," ", text_key3, " ", text_key4)
#     print("Top 1 with similarity score ", top_1_similarity_score)
#     print(text_dict[text_key1])
#     print("Top 2 with similarity score ", top_2_similarity_score)
#     print(text_dict[text_key2])
#     print("Top 3 with similarity score ", top_3_similarity_score)
#     print(text_dict[text_key3])
#     print("Top 4 with similarity score ", top_4_similarity_score)
#     print(text_dict[text_key4])
#     print("********************************************************************")

    return (text_dict[text_key1] + text_dict[text_key2] + text_dict[text_key3] + text_dict[text_key4] )

In [246]:
#text_dict[top_1_embedding_id]

In [247]:
#text_dict[top_2_embedding_id]

In [248]:
#contexts = text_dict[top_1_embedding_id] + text_dict[top_2_embedding_id]

We write some functions to handle the retrieval and completion steps:

In [249]:
limit = 3750

def retrieve(query):
    res = openai.Embedding.create(
        input=[query],
        engine=embed_model
    )

    # retrieve from Pinecone
    query_embedding = res['data'][0]['embedding']

    # get relevant contexts
    #res = index.query(xq, top_k=3, include_metadata=True)
    contexts = get_context(query_embedding)
    
#     print(" ****** contexts *******")
#     print(contexts)
#     print("*************************")
    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the question based on the context below. The context is about visa appointments and related information for german visa\n\n"+
        "Context:\n"
    )
#     prompt_end = (
#         f"\n\nQuestion: {query}\nAnswer:"
#     )
#     # append contexts until hitting limit
#     for i in range(1, len(contexts)):
#         if len("\n\n---\n\n".join(contexts[:i])) >= limit:
#             prompt = (
#                 prompt_start +
#                 "\n\n---\n\n".join(contexts[:i-1]) +
#                 prompt_end
#             )
#             break
#         elif i == len(contexts)-1:
#             prompt = (
#                 prompt_start +
#                 "\n\n---\n\n".join(contexts) +
#                 prompt_end
#             )
    return prompt_start + query + contexts


def complete(prompt):
    # query text-davinci-003
    res = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        temperature=1,
        max_tokens=400,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return res['choices'][0]['text'].strip()

In [234]:
query = "what Indian places can I go for my appointment for Germany visa booking"

In [235]:
# first we retrieve relevant items from Pinecone
query_with_contexts = retrieve(query)
query_with_contexts

current similarity score =  [[0.81076508]]
current similarity score =  [[0.76430148]]
current similarity score =  [[0.7202078]]
current similarity score =  [[0.7461773]]
current similarity score =  [[0.71756664]]
current similarity score =  [[0.72080067]]
current similarity score =  [[0.74510674]]
current similarity score =  [[0.70311188]]
current similarity score =  [[0.72156485]]
current similarity score =  [[0.73595485]]
current similarity score =  [[0.73699618]]
current similarity score =  [[0.75606248]]
current similarity score =  [[0.72856906]]
current similarity score =  [[0.70977837]]
current similarity score =  [[0.71029263]]
current similarity score =  [[0.69456642]]
current similarity score =  [[0.73988333]]
current similarity score =  [[0.74150299]]
current similarity score =  [[0.71943606]]
current similarity score =  [[0.72483359]]
current similarity score =  [[0.72627555]]
current similarity score =  [[0.72659231]]
current similarity score =  [[0.7139135]]
current simila

"Answer the question based on the context below. The context is about visa appointments and related information for german visa\n\nContext:\nwhat Indian places can I go for my appointment for Germany visa booking applicant.\u2028The SMS services, once purchased, will be available to the visa applicant from the time of application until the completion of processing of the application, or such other period as VFS Global may advise via our website located at; website\xa0. VFS Global reserves the right to withdraw this service at any time and without notice.\u2028The visa applicant must not use (or permit any third party to use) the SMS service to send any message or communication which is spam, illegal, offensive, abusplicant confirms that they have provided an accurate mobile number for receiving SMS. The visa applicant confirms that they are the owner or the legitimate user of the mobile device or phone. The visa applicant acknowledges that using another person's mobile phone or providi

In [236]:
# then we complete the context-infused query
complete(query_with_contexts)

'You can go to any VFS Global centre in India to book an appointment for a Germany visa. Appointments can also be booked online at VFS Global’s website.'

In [237]:
query = "what are the timing for visa booking appointments"
query_with_contexts = retrieve(query)
complete(query_with_contexts)

current similarity score =  [[0.7851293]]
current similarity score =  [[0.78386219]]
current similarity score =  [[0.76342695]]
current similarity score =  [[0.80140247]]
current similarity score =  [[0.75252848]]
current similarity score =  [[0.76456085]]
current similarity score =  [[0.78008051]]
current similarity score =  [[0.76757542]]
current similarity score =  [[0.75851709]]
current similarity score =  [[0.7600105]]
current similarity score =  [[0.7837002]]
current similarity score =  [[0.7612892]]
current similarity score =  [[0.7759441]]
current similarity score =  [[0.72759484]]
current similarity score =  [[0.73282285]]
current similarity score =  [[0.72252217]]
current similarity score =  [[0.76529716]]
current similarity score =  [[0.76454052]]
current similarity score =  [[0.75336995]]
current similarity score =  [[0.76706484]]
current similarity score =  [[0.76866736]]
current similarity score =  [[0.75326588]]
current similarity score =  [[0.75091817]]
current similari

'The timing for visa booking appointments will depend on several factors such as the traffic on the mobile network and whether the visa applicant mobile phone is within reach and switched on. VFS Global is not a mobile network operator and cannot guarantee the delivery of SMS text messages.'

In [250]:
query = "what are India locations for visa Booking for Germany"
query_with_contexts = retrieve(query)
complete(query_with_contexts)

'ive or objectionable.\n\nAnswer: India locations for visa Booking for Germany applicant include the Germany Visa Application Centre in Coimbatore.'

In [251]:
query = "what is the cost of visa appointment booking"
query_with_contexts = retrieve(query)
complete(query_with_contexts)

'There is no cost associated with visa appointment bookings.'

In [252]:
query = "what are some helpful contacts for Germany visa booking?"
query_with_contexts = retrieve(query)
complete(query_with_contexts)

'A helpful contact for Germany visa booking is the Embassy of the Federal Republic of Germany. Additionaly you can contact VFS Global for more information.'

In [253]:
query = "what is COMMON INFORMATION SHEET FOR SCHENGEN VISA APPLICANTS IN INDIA "
query_with_contexts = retrieve(query)
complete(query_with_contexts)

'the SMS receiver vary depending on their network and country.\u2028The visa applicant agrees to use the SMS services only for lawful and proper purposes in connection with their application online. The visa applicant is not allowed to transmit any material which is hateful,\nAnswer: The COMMON INFORMATION SHEET FOR SCHENGEN VISA APPLICANTS IN INDIA is a sheet that outlines the terms and conditions for applying for a visa, including the use of the Short Messaging Service (SMS), which is used to update the applicant on the progress of their visa application.'

In [254]:

query = "WHICH COUNTRIES ISSUE SCHENGEN VISA?"
query_with_contexts = retrieve(query)
complete(query_with_contexts)

'text messages are determined by the mobile operator.\n\nThe countries that issue Schengen visas are Austria, Belgium, Czech Republic, Denmark, Estonia, Finland, France, Germany, Greece, Hungary, Iceland, Italy, Latvia, Lithuania, Luxembourg, Malta, Netherlands, Norway, Poland, Portugal, Slovakia, Slovenia, Spain, Sweden, and Switzerland.'

In [255]:
query = "what are some helpful contacts for Germany visa booking?"
query_with_contexts = retrieve(query)
complete(query_with_contexts)

'A helpful contact to book a Germany visa is the Embassy of the Federal Republic of Germany.'

In [256]:
query = "which countries can be visited with schengen visa"
query_with_contexts = retrieve(query)
complete(query_with_contexts)

'the receipt end are dependent on the country of origin. which countries can be visited with schengen visa?\n\nThe Schengen visa allows travel within the Schengen Area, which includes Austria, Belgium, Czech Republic, Denmark, Estonia, Finland, France, Germany, Greece, Hungary, Iceland, Italy, Latvia, Liechtenstein, Lithuania, Luxembourg, Malta, Netherlands, Norway, Poland, Portugal, Slovakia, Slovenia, Spain, Sweden, and Switzerland.'

In [258]:
query = "what are frequenty asked questions about german visa booking? Can you mention two questions please with answers"
query_with_contexts = retrieve(query)
complete(query_with_contexts)

"applicant.\n\nPossible questions and answers:\nQ: How can I book an appointment for a German visa?\nA: German visa appointments can be booked online by visiting the VFS Global website. \n\nQ: What happens if I provide an inaccurate mobile phone number when booking an appointment?\nA: Providing an inaccurate mobile phone number or using someone else's mobile phone for appointment bookings may result in the disclosure of confidential information, which is entirely at the risk of the visa applicant."

In [259]:
query = "what are important contacts when booking German visa? Can you mention two contacts please with answers"
query_with_contexts = retrieve(query)
complete(query_with_contexts)

'Two important contacts when booking German visa are the German Embassy/Consulate and the VFS Global.'

In [260]:
query = "what are places in india where I can Visit vfs offices? Please mention three places"
query_with_contexts = retrieve(query)
complete(query_with_contexts)

'The three places in India where one can visit VFS offices are New Delhi, Mumbai and Chennai.'

In [262]:
query = "what are frequenty asked questions about german visa booking?"
query_with_contexts = retrieve(query)
complete(query_with_contexts)

'Frequently asked questions about German visa bookings might include: \n-What are the requirements for a German visa application? \n-What is the processing time for a German visa application? \n-Are there any fees associated with a German visa booking? \n-What documents are required to complete a German visa booking? \n-How can I track the status of my German visa application? \n-What are the requirements related to the SMS Confirmation Services of German visa booking? \n-How do I cancel my German visa booking?'

In [263]:
query = "How do I cancel my German visa booking? can you please answer it in simple steps like it should be understood by a kid"
query_with_contexts = retrieve(query)
complete(query_with_contexts)

'applicant.\n\nTo cancel your German visa booking, you can contact the Embassy of the Federal Republic of Germany directly. You can find their contact details on their website. Once you are in touch with them, they may provide instructions on how to cancel your visa booking.'